In [1]:
!pip install transformers sentencepiece


In [2]:
from transformers import MarianMTModel, MarianTokenizer

model_name = "Helsinki-NLP/opus-mt-zh-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/805k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/807k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.62M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [3]:
# 輸入要翻譯的中文句子
text = "今天天氣非常好，適合出門散步。"

# 將輸入文字轉換為模型所需的格式
inputs = tokenizer(text, return_tensors="pt")

# 使用模型生成翻譯結果
translated_tokens = model.generate(**inputs)

# 解碼翻譯結果
translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

print("翻譯結果：", translated_text)


翻譯結果： It's a beautiful day for a walk.


In [9]:
# Define the input (idiom)
input_text = "画蛇添足"  # Idiom: "to add feet to a snake"
inputs = tokenizer(input_text, return_tensors="pt")

# Generate translation with output scores for self-confidence
translated = model.generate(**inputs, output_scores=True, return_dict_in_generate=True)
translated_text = tokenizer.decode(translated.sequences[0], skip_special_tokens=True)

# Compute average token confidence
# Use the minimum length between logits and (pred_ids - 1) to avoid index error.
logits = translated.scores  # list of tensors (one per decoding step)
pred_ids = translated.sequences[0]

n_steps = min(len(logits), len(pred_ids) - 1)
token_confidences = []

for i in range(n_steps):
    token_id = pred_ids[i+1]  # each logit corresponds to token at position i+1
    probs = F.softmax(logits[i], dim=-1)
    token_prob = probs[0, token_id].item()
    token_confidences.append(token_prob)

average_confidence = sum(token_confidences) / len(token_confidences) if token_confidences else 0

print("Input:", input_text)
print("Model Output:", translated_text)
print("Average Token Confidence:", average_confidence)

Input: 画蛇添足
Model Output: Draw the snake's feet.
Average Token Confidence: 0.24106994811154436


In [12]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import torch.nn.functional as F

# Load the summarization model and tokenizer
model_name = "google/pegasus-xsum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Define the input text
input_text = ("The report details how a major company increased its quarterly revenue by 25% "
              "due to new product launches and strategic investments. However, it also noted "
              "rising costs and market volatility as potential risks.")

# Tokenize and generate summary with scores
inputs = tokenizer(input_text, return_tensors="pt", truncation=True)
output = model.generate(**inputs,
                        max_length=60,
                        output_scores=True,
                        return_dict_in_generate=True)
summary = tokenizer.decode(output.sequences[0], skip_special_tokens=True)

# Calculate average token confidence safely
logits = output.scores  # list of tensors (one per decoding step)
pred_ids = output.sequences[0]
token_confidences = []
# Ensure that we only iterate over available token scores and valid token indices.
num_steps = min(len(logits), len(pred_ids) - 1)
for i in range(num_steps):
    token_id = pred_ids[i+1]  # scores correspond to tokens 1..n
    prob = F.softmax(logits[i], dim=-1)[0, token_id].item()
    token_confidences.append(prob)
avg_confidence = sum(token_confidences) / len(token_confidences) if token_confidences else 0.0

print("Input:", input_text)
print("Summary:", summary)
print("Average Token Confidence:", avg_confidence)


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Input: The report details how a major company increased its quarterly revenue by 25% due to new product launches and strategic investments. However, it also noted rising costs and market volatility as potential risks.
Summary: The pharmaceutical and medical device industry has seen strong growth over the past few years, according to a new report by the World Health Organization (WHO).
Average Token Confidence: 0.1673746878144385


In [14]:

# Load the PEGASUS-XSumm model and tokenizer
model_name = "google/pegasus-xsum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Define an English input text containing multiple key pieces of information
input_text = (
    "Recently, the government announced a series of measures to reduce urban pollution, "
    "including stricter emission controls and the expansion of green spaces. "
    "It is predicted that these measures will lower air pollution by 30% within one year. "
    "Environmental groups have praised the initiative, while some industry representatives have raised concerns about potential negative economic impacts."
)

# (Optional) Define an expected summary for reference
expected_summary = (
    "The government announced new measures to reduce urban pollution through stricter emission controls "
    "and expanding green spaces, aiming to lower pollution by 30% within a year. "
    "Environmental groups support the plan, but industry worries about economic downsides."
)

# Convert the input text into the format required by the model
inputs = tokenizer(input_text, return_tensors="pt", truncation=True)

# Generate a summary with beam search and output the token scores
output = model.generate(
    **inputs,
    max_length=60,
    num_beams=5,
    output_scores=True,
    return_dict_in_generate=True
)

# Decode the generated token IDs to a human-readable summary
generated_summary = tokenizer.decode(output.sequences[0], skip_special_tokens=True)

# Compute the average token confidence (approximate self-confidence)
# Note: output.scores contains a list of logits (one per decoding step)
token_confidences = []
for i in range(num_steps):
    token_id = pred_ids[i+1]  # scores correspond to tokens 1..n
    prob = F.softmax(logits[i], dim=-1)[0, token_id].item()
    token_confidences.append(prob)
avg_confidence = sum(token_confidences) / len(token_confidences) if token_confidences else 0.0

avg_confidence = sum(token_confidences) / len(token_confidences)

# Print the results
print("Input Text:\n", input_text)
print("\nExpected Summary:\n", expected_summary)
print("\nGenerated Summary:\n", generated_summary)
print("\nAverage Token Confidence:", avg_confidence)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Input Text:
 Recently, the government announced a series of measures to reduce urban pollution, including stricter emission controls and the expansion of green spaces. It is predicted that these measures will lower air pollution by 30% within one year. Environmental groups have praised the initiative, while some industry representatives have raised concerns about potential negative economic impacts.

Expected Summary:
 The government announced new measures to reduce urban pollution through stricter emission controls and expanding green spaces, aiming to lower pollution by 30% within a year. Environmental groups support the plan, but industry worries about economic downsides.

Generated Summary:
 The Chinese government has announced a series of measures to reduce urban pollution, including stricter emission controls and the expansion of green spaces.

Average Token Confidence: 0.1673746878144385
